# Day 4 : AI Flight Price Assistant

Features:
- Fetch mock airline prices
- Save them into SQLite
- Update specific airline prices
- Show stored prices
- Optionally track price changes

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3
from datetime import datetime

In [2]:
# Initialization

load_dotenv(override=True)

openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
if openrouter_api_key:
    print(f"OpenAI API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=openrouter_api_key)


OpenAI API Key exists and begins sk-or-v1


In [51]:
# setup sqlite db

DB = "airline_prices.db"

with sqlite3.connect(DB) as conn:
    create_table = """
    CREATE TABLE flights (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    airline TEXT,
    origin TEXT,
    destination TEXT,
    price REAL,
    last_updated TEXT,
    UNIQUE(origin, destination, airline)
    );
    """
    cursor = conn.cursor()
    cursor.execute(create_table)

In [52]:

# mock data for setting up the db

original_flight_prices = [
    {"airline": "Kenya Airways", "origin": "Nairobi", "destination": "London", "price": 920.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Emirates", "origin": "Nairobi", "destination": "London", "price": 870.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Qatar Airways", "origin": "Nairobi", "destination": "London", "price": 890.0, "last_updated": datetime.now().isoformat()},
    {"airline": "British Airways", "origin": "Nairobi", "destination": "London", "price": 940.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Turkish Airlines", "origin": "Nairobi", "destination": "London", "price": 880.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Kenya Airways", "origin": "Nairobi", "destination": "Dubai", "price": 500.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Emirates", "origin": "Nairobi", "destination": "Dubai", "price": 480.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Qatar Airways", "origin": "Nairobi", "destination": "Dubai", "price": 490.0, "last_updated": datetime.now().isoformat()},  
    {"airline": "Kenya Airways", "origin": "Nairobi", "destination": "New York", "price": 1300.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Emirates", "origin": "Nairobi", "destination": "New York", "price": 1280.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Qatar Airways", "origin": "Nairobi", "destination": "New York", "price": 1295.0, "last_updated": datetime.now().isoformat()},
    {"airline": "British Airways", "origin": "Nairobi", "destination": "New York", "price": 1320.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Kenya Airways", "origin": "Nairobi", "destination": "Johannesburg", "price": 350.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Emirates", "origin": "Nairobi", "destination": "Johannesburg", "price": 340.0, "last_updated": datetime.now().isoformat()},
    {"airline": "South African Airways", "origin": "Nairobi", "destination": "Johannesburg", "price": 360.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Kenya Airways", "origin": "Mombasa", "destination": "Dubai", "price": 520.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Emirates", "origin": "Mombasa", "destination": "Dubai", "price": 500.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Qatar Airways", "origin": "Mombasa", "destination": "Dubai", "price": 510.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Kenya Airways", "origin": "Mombasa", "destination": "London", "price": 950.0, "last_updated": datetime.now().isoformat()},
    {"airline": "Emirates", "origin": "Mombasa", "destination": "London", "price": 920.0, "last_updated": datetime.now().isoformat()},
]


In [ ]:
for flight in original_flight_prices:
    print(flight)

In [53]:
def insert_ticket_price(flight_prices):

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()

        data_to_insert = [
            (f["airline"], f["origin"], f["destination"], f["price"], f["last_updated"])
            for f in flight_prices
        ]

        cursor.executemany("""
            INSERT OR IGNORE INTO flights 
            (airline, origin, destination, price, last_updated)
            VALUES (?, ?, ?, ?, ?)
        """, data_to_insert)

        conn.commit()


In [54]:
# now insert the mock data into the db
insert_ticket_price(original_flight_prices)

In [55]:
def get_ticket_price(origin, destination):
    print(f"DATABASE TOOL CALLED: Getting prices for {origin} to {destination}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'SELECT airline, price FROM flights WHERE lower(origin)=? AND lower(destination)=?',
            (origin.lower(), destination.lower())
        )
        results = cursor.fetchall()
    
    results_list = [
        {"airline": airline, "origin": origin, "destination": destination, "price": price}
        for airline, price in results
    ]
    print(results_list)

    return results_list


In [ ]:
ticket_prices = get_ticket_price("Nairobi", "London")
ticket_prices

In [57]:
def set_ticket_price(origin, destination, airline, price):
    print(f"DATABASE TOOL CALLED: Setting prices for {origin} to {destination}", flush=True)
    now = datetime.now().isoformat()

    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO flights (airline, origin, destination, price, last_updated)
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(origin, destination, airline)
            DO UPDATE SET
                price = excluded.price,
                last_updated = excluded.last_updated
        """, (airline, origin, destination, price, now))
        conn.commit()

In [64]:
TOOL_FUNCTIONS = {
    "get_ticket_price": get_ticket_price,
    "set_ticket_price": set_ticket_price,
}
def handle_tool_calls(message):
    responses = []

    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        match tool_name:
            case name if name in TOOL_FUNCTIONS:
                # Call the corresponding function with the unpacked arguments
                result = TOOL_FUNCTIONS[name](**arguments)
                responses.append({
                    "role": "tool",
                    "content": json.dumps(result),
                    "tool_call_id": tool_call.id
                })
            case _:
                responses.append({
                    "role": "tool",
                    "content": f"Tool '{tool_name}' not recognized.",
                    "tool_call_id": tool_call.id
                })

    return responses

In [65]:
system_message = """
You are a helpful assistant for an airline called FlightAI.
- Give short, courteous answers.
- Always be accurate and only use stored flight price data.
- Ensure that the response is a markdown formatted table. with the origin, destination, airline, and price.
- If more than one airline is available get the cheapest three sorted by price.
- You can provide ticket prices for routes or update prices if the user provides new information.
- If you do not have data for a route, say so.
- Do not fabricate airline prices or routes.
"""


In [66]:
# Tool dictionary for getting ticket prices
get_ticket_price_function = {
    "name": "get_ticket_price",
    "description": "Get all flight prices for a given origin and destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "The departure city",
            },
            "destination": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["origin", "destination"],
        "additionalProperties": False
    }
}

# Tool dictionary for setting/updating a ticket price
set_ticket_price_function = {
    "name": "set_ticket_price",
    "description": "Update the price of a ticket for a specific airline, origin, and destination.",
    "parameters": {
        "type": "object",
        "properties": {
            "origin": {
                "type": "string",
                "description": "The departure city",
            },
            "destination": {
                "type": "string",
                "description": "The arrival city",
            },
            "airline": {
                "type": "string",
                "description": "The airline of the flight",
            },
            "price": {
                "type": "number",
                "description": "The updated price of the ticket",
            },
        },
        "required": ["origin", "destination", "airline", "price"],
        "additionalProperties": False
    }
}


In [67]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": get_ticket_price_function}, {"type": "function", "function": set_ticket_price_function}]

In [68]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()